In [22]:
################
# Setup DuckDB #
################

import duckdb
from duckdb import SQLExpression

duck = duckdb.connect()  # in-memory DB

In [23]:
# Check connected realms

source = '/Users/julien.leloup/Personal/ShadowDrive/wow-analytics-data/eu/global/connected_realms.parquet'

connected_realms = duck.read_parquet(source)

realm = connected_realms.filter(SQLExpression("realm_slugs LIKE '%elune%'"))
realm.show()

┌────────────────────┬───────────┬─────────────┬─────────────────┬────────────┬────────────────────┬────────────────────┐
│ connected_realm_id │ has_queue │ status_type │ population_type │ num_realms │    realm_names     │    realm_slugs     │
│       int16        │  boolean  │   varchar   │     varchar     │    int8    │      varchar       │      varchar       │
├────────────────────┼───────────┼─────────────┼─────────────────┼────────────┼────────────────────┼────────────────────┤
│               1315 │ false     │ UP          │ MEDIUM          │          2 │ Elune, Varimathras │ elune, varimathras │
└────────────────────┴───────────┴─────────────┴─────────────────┴────────────┴────────────────────┴────────────────────┘



In [ ]:
# Check auctions

server_id = "1315"

source = '/Users/julien.leloup/Personal/ShadowDrive/wow-analytics-data/eu/auctions/*/*/{server_id}/*.parquet'

last_auctions = duck.read_parquet(source)
last_auctions.show()

┌───────────────────────────┬────────────┬─────────┬──────────┬────────────┬───────────┬───────────┬────────────────────┬────────────┬─────────────┬─────────────┬────────────┬─────────────┬─────────────┬─────────────┬─────────────┬─────────────┬─────────────┬─────────────┬───────┬─────────────┬─────────────┬─────────────┬─────────────┬─────────────┬─────────────┬─────────────┬─────────────┬─────────────┐
│      fetch_timestamp      │ auction_id │ item_id │ quantity │ unit_price │  buyout   │ time_left │ connected_realm_id │ modifier_6 │ modifier_28 │ bonus_lists │ modifier_9 │ modifier_44 │ modifier_29 │ modifier_38 │ modifier_39 │ modifier_40 │ modifier_41 │ modifier_42 │  bid  │ modifier_43 │ modifier_30 │ modifier_50 │ modifier_49 │ modifier_48 │ modifier_51 │ modifier_45 │ modifier_46 │ modifier_47 │
│       timestamp_ns        │   int64    │  int32  │  int16   │   int64    │   int64   │  varchar  │       int16        │   double   │   double    │   varchar   │   double   │   double